# Used for predicting ALSFRS_slope
see https://www.synapse.org/#!Synapse:syn2873386/wiki/ .
We assumed data is vectorized + clustered + 6 features were selected

In [7]:
from IPython.display import display

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from vectorizing_funcs import *
from modeling_funcs import *

## Revectorize the selected data
We now reload the metadata and the 6 attributes selected per cluster

In [9]:
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )
train_data_means = pickle.load( open('../all_data_means.pickle', 'rb') )
train_data_std = pickle.load( open('../all_data_std.pickle', 'rb') )
best_features_per_cluster = pickle.load( open('../best_features_per_cluster.pickle', 'rb') )


df = pd.read_csv('../all_data_selected.csv', sep='|', index_col=False, dtype="unicode")
vectorized, _ = vectorize(df, all_feature_metadata)
normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
print normalized.shape
normalized.head()

(2424, 157)


,family_ALS_hist_last,Hematocrit_mean,Hematocrit_pct_diff,Hematocrit_last,Hematocrit_mean_slope,weight_mean,weight_pct_diff,weight_last,weight_mean_slope,Sodium_mean,...,Absolute Basophil Count_last,Absolute Basophil Count_mean_slope,Protein_mean,Protein_pct_diff,Protein_last,Protein_mean_slope,ALT(SGPT)_mean,ALT(SGPT)_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7799,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
normalized.describe().T.sort("mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
respiratory_last,2424,1.888580e-01,0.564736,-3.049000,0.000000,0.566232,0.566232,0.566232
respiratory_mean,2424,1.768065e-01,0.607240,-3.917850,0.000000,0.076491,0.647111,0.647111
respiratory_mean_slope,2424,6.371082e-02,0.722169,-4.473866,0.000000,0.056652,0.056652,21.199067
weight_last,2424,5.714988e-02,0.813019,-2.152398,-0.282487,0.000000,0.321272,4.522234
weight_pct_diff,2424,5.460839e-02,0.710178,-10.263972,-0.048791,0.000000,0.200132,5.422245
respiratory_pct_diff,2424,5.347888e-02,0.770415,-4.121627,0.000000,0.004858,0.004858,25.680768
weight_mean,2424,5.152330e-02,0.813480,-2.163820,-0.295905,0.000000,0.330740,4.617477
weight_mean_slope,2424,3.987349e-02,0.774323,-8.099138,-0.067200,0.000000,0.114570,7.515461
Creatinine_pct_diff,2424,1.225406e-02,0.924509,-0.033857,-0.020545,0.000000,0.000000,45.508135
mouth_mean,2424,3.149662e-15,0.999794,-4.755964,-0.608246,0.428683,0.774327,0.774327


In [13]:
slope = pd.read_csv('../all_slope.csv', sep = '|', index_col="SubjectID")
clusters = pd.read_csv('../all_kmeans_clusters.csv', sep = '|', index_col="SubjectID")

clusters.index = clusters.index.astype(str)
slope.index = slope.index.astype(str)
normalized.index = normalized.index.astype(str)

X = normalized.join(clusters)
Y = slope.join(clusters)

print Y.shape, X.shape, clusters.shape
display(X.head(3))
display(Y.head(3))


(2424, 2) (2424, 158) (2424, 1)


,family_ALS_hist_last,Hematocrit_mean,Hematocrit_pct_diff,Hematocrit_last,Hematocrit_mean_slope,weight_mean,weight_pct_diff,weight_last,weight_mean_slope,Sodium_mean,...,Absolute Basophil Count_mean_slope,Protein_mean,Protein_pct_diff,Protein_last,Protein_mean_slope,ALT(SGPT)_mean,ALT(SGPT)_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope,cluster
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5588,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,ALSFRS_slope,cluster
SubjectID,,
533,-0.965608,0
649,-0.921717,1
1234,-0.914787,0


## Train a prediction model per cluster

In [14]:
model_per_cluster = get_model_per_cluster(X, Y)
    

cluster: 0 size: (826L,)
	 RMS error (0 is perfect): 0.56
	 explained variance score (1 is perfect): 0.18
cluster: 1 size: (1598L,)
	 RMS error (0 is perfect): 0.56
	 explained variance score (1 is perfect): 0.21


In [21]:
print model_per_cluster[0]["model"].coef_

[  0.00000000e+00  -1.39542163e+12  -1.30587448e+13  -1.02692549e+13
   4.09614774e+13   8.36052533e+12  -5.46064651e+12   1.42588332e+13
  -1.41972149e+13  -1.41850718e+13  -1.47782912e+12  -8.42770248e+11
   3.90043412e+11  -7.86577058e-02  -1.31988878e-02   2.72781506e-02
  -1.72701177e-02   2.07475655e+12   4.36390172e+11   2.45697803e+12
  -1.54411212e+11  -5.78146855e+11  -8.67547879e+11  -5.06629632e+11
   4.36762367e+11  -2.03106837e+10   2.76078571e+11   7.24371310e+10
  -4.26009700e+10  -1.15833682e+10   8.21623912e+08  -3.48149980e+07
   1.87827441e+07  -6.54723634e-02   3.22148389e-03   2.71809630e-02
   1.49324349e-02  -8.57495179e-16   1.00753654e-14  -2.46483357e-15
  -2.36398864e-14  -9.93635665e-15  -2.11518236e-15   1.93933778e-14
  -5.68952225e-15  -9.35425907e-15  -3.60127802e-15   6.33658018e-16
  -7.02556489e-15   2.97624370e-15  -5.43029722e-16   2.18972527e-16
   1.50378748e-16   2.43761598e-16  -7.27320521e-16   2.92663389e-16
  -4.49384759e-16  -1.00137357e-15

In [15]:
with open("../model_per_cluster.pickle", "wb") as output_file:
    pickle.dump(model_per_cluster, output_file)


## Apply the model on both `train` and `test`

In [16]:

for t in ['all', 'test']:
    df = pd.read_csv('../' + t + '_data_selected.csv', sep='|', index_col=False)
    vectorized, _ = vectorize(df, all_feature_metadata)
    normalized, _ = normalize(vectorized, all_feature_metadata, train_data_means, train_data_std)
    
    clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col=0)

    clusters.index = clusters.index.astype(str)
    normalized.index = normalized.index.astype(str)

    X = normalized.join(clusters)
    pred = X.apply(apply_model, args=[model_per_cluster], axis = 1)
    pred.to_csv('../' + t + '_prediction.csv',sep='|')

pred.head()

,cluster,prediction
SubjectID,,
751103,0,-0.86851
753161,0,-0.86851
754593,0,-0.86851
756127,0,-0.86851
757731,0,-0.86851
